In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/MyDrive/MP_DATA.zip' -d '/content/drive/MyDrive/'


In [7]:
!pip install mediapipe

In [8]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [9]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [10]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results


In [11]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [13]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [15]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)  #33*4 extra 1 is for visibilty only pose has this
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)    #468*3
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)  #21 keypoints and 3 dimension x,y,z
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])


In [16]:
#SETTING DIRECTORY
#Path for exported_data i.e numpy arrays
DATA_PATH = os.path.join('MP_DATA')

#subsequent actions
actions = np.array(['Helloh','Dankz','ilubYu','Super','Namaste_Vanakkam','Okay Dude'])

no_sequences = 30  # 30 number of vids of data
sequence_length = 30 #each of 30frames per sec in length


In [11]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

In [17]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):


              # Read feed
              ret, frame = cap.read()

              # Make detections
              image, results = mediapipe_detection(frame, holistic)
              print(results)

              # Draw landmarks
              draw_styled_landmarks(image, results)

              #collection logic with breaks
              if frame_num ==  0:         #0th frame
               cv2.putText(image,'STARTING COLLECTION',(120,200),
                           cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),4,cv2.LINE_AA)
               cv2.putText(image,'Collecting frames for {} Video number{}'.format(action,sequence),(15,12),
                           cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
              else:
                 cv2.putText(image,'Collecting frames for {} Video Number{}'.format(action,sequence),(15,12),
                             cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)

                 #export keypoints
              keypoints = extract_keypoints(results)
              npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
              np.save(npy_path,keypoints)

              # Show to screen
              cv2.imshow('OpenCV Feed', image)

            # Break gracefully
              if cv2.waitKey(10) & 0xFF == ord('q'):
                 break
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

In [18]:
!pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/77/85/bff3a1e818ec6aa3dd466ff4f4b0a727db9fdb41f2e849747ad902ddbe95/scikit_learn-1.3.0-cp311-cp311-win_amd64.whl.metadata
  Using cached scikit_learn-1.3.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/06/15/e73734f9170b66c6a84a0bd7e03586e87e77404e2eb8e34749fc49fa43f7/scipy-1.11.2-cp311-cp311-win_amd64.whl.metadata
  Using cached scipy-1.11.2-cp311-cp311-win_amd64.whl.metadata (59 kB)
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712

In [13]:
cap.release()
cv2.destroyAllWindows()

In [1]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/9e/b8/ed5f794359d05cd0bffb894c6418da87b93016ee17b669d55c45d1bd5d5b/tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata (2.6 kB)
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages/2f/2f/3c84f675931ce3bcbc7e23acbba1e5d7f05ce769adab48322de57a9f5928/tensorflow_intel-2.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow_intel-2.13.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Obtaining dependency information for h5py>=2.9.0 from https://files.pythonhosted.org/packages/d1/93/0f4cf5058095d749d464e4f770d2bf339930e5f3374331f0d2fa6ddfbf28/h5py-3.9.0-cp311-cp311-win_amd64.whl.metadata
  Using c

In [2]:
from tensorflow.keras.utils import to_categorical

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
label_map = {label:num for num,label in enumerate(actions)}

In [38]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [39]:
sequences,labels = [],[]
for action in actions:
  for sequence in range(no_sequences):
    window = []
    for frame_num in range(sequence_length):
      res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
      window.append(res)
    sequences.append(window)
    labels.append(label_map[action])

In [40]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [41]:
y

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1]])

In [42]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.05)

**LSTM Neural Network**

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [44]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [45]:
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu', input_shape=(30,1662)))   #30frames, 1662 keypoints
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))


In [46]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy']) #for multi class use categ cross entropy

In [47]:
model.fit(X_train,y_train,epochs=1500,callbacks=[tb_callback])

Epoch 1/1500
6/6 [==============================] - 5s 133ms/step - loss: 3.0111 - categorical_accuracy: 0.1579
Epoch 2/1500
6/6 [==============================] - 1s 138ms/step - loss: 2.6761 - categorical_accuracy: 0.1637
Epoch 3/1500
6/6 [==============================] - 1s 139ms/step - loss: 1.9037 - categorical_accuracy: 0.1813
Epoch 4/1500
6/6 [==============================] - 1s 134ms/step - loss: 2.0522 - categorical_accuracy: 0.1404
Epoch 5/1500
6/6 [==============================] - 1s 152ms/step - loss: 1.8294 - categorical_accuracy: 0.1754
Epoch 6/1500
6/6 [==============================] - 1s 224ms/step - loss: 1.8105 - categorical_accuracy: 0.1988
Epoch 7/1500
6/6 [==============================] - 1s 228ms/step - loss: 2.0602 - categorical_accuracy: 0.1579
Epoch 8/1500
6/6 [==============================] - 1s 226ms/step - loss: 1.7881 - categorical_accuracy: 0.2807
Epoch 9/1500
6/6 [==============================] - 1s 225ms/step - loss: 1.7967 - categorical_accuracy:

KeyboardInterrupt: ignored

**Make Predictions**

In [48]:
res= model.predict[X_test]

TypeError: ignored

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

**Save Model**

In [49]:
model.save('MediapipeModel.h5')

In [22]:
from sklearn.metrics import accuracy_score


In [ ]:
yhat = model.predict(X_test)

In [ ]:
yactual= np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(yactual,yhat)

In [23]:
accuracy_score(ytrue,yhat)

NameError: name 'ytrue' is not defined

In [71]:
# Update the colors list with two more colors
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245), (255, 0, 0), (0, 255, 0),(255,165,0)]

# Function to visualize probabilities
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    return output_frame

In [51]:
from scipy import stats

**Test in Real time**

In [62]:
sequence=[]
sentence=[]
threshold = 0.4

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    ret,frame = cap.read()

    image,results = mediapipe_detection(frame,holistic)
    print(results)

    draw_styled_landmarks(image,results)

    #Prediction logic code
    keypoints = extract_keypoints(results)
    sequence.append(keypoints)
    sequence = sequence[:-30]

    if len(sequence) ==30:
      res = model.predict(np.expand_dims(sequence,axis=0))[0]
      print(actions[np.argmax(res)])


      #Visulaisation prediction logic
    if res[np.argmax(res)] > threshold:
      if len(sentence) > 0:
        if actions[np.argmax(res)] != sentence[-1]:
          sentence.append(actions[np.argmax(res)])
      else:
          sentence.append(actions[np.argmax(res)])
    if len(sentence) > 5:
      sentence = sentence[-5:]

    #Visualise Probability
    image = prob_viz(res,actions,image,actions)
    # cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
    cv2.putText(image, ''.join(sentence),(3,30),
                  cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('OpenCV Feed',image)

    if cv2.waitKey(10) & 0xFF ==ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>


error: OpenCV(4.8.0) :-1: error: (-5:Bad argument) in function 'rectangle'
> Overload resolution failed:
>  - Scalar value for argument 'color' is longer than 4
>  - Scalar value for argument 'color' is longer than 4
>  - Can't parse 'rec'. Expected sequence length 4, got 2
>  - Can't parse 'rec'. Expected sequence length 4, got 2


In [46]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('C:/Users/jaikr/Downloads/MediapipeModel.h5')

# Define the threshold for prediction
threshold = 0.4

# Define your functions for visualization and other tasks
# (mediapipe_detection, draw_styled_landmarks, extract_keypoints, prob_viz)

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        image, results = mediapipe_detection(frame, holistic)
        print(results)

        draw_styled_landmarks(image, results)

        # Prediction logic code
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]  # Maintain the last 30 frames

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            # Visualization prediction logic
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0:
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5:
                sentence = sentence[-5:]

            # # Visualize Probability
            # image = prob_viz(res, actions, image, colors)
            # cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
            # cv2.putText(image, ''.join(sentence), (3, 30),
            #             cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


<class 'mediapipe.python.solution_base.SolutionOutputs'>


TypeError: insert expected 2 arguments, got 1

In [75]:
cap.release()
cv2.destroyAllWindows()

In [50]:
 #Load the trained model
# model = load_model('C:/Users/jaikr/Downloads/MediapipeModel.h5')
sequence=[]
sentence=[]
# Define the threshold for prediction
threshold = 0.4

# Define your functions for visualization and other tasks
# (mediapipe_detection, draw_styled_landmarks, extract_keypoints, prob_viz)

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)



with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    ret,frame = cap.read()

    image,results = mediapipe_detection(frame,holistic)
    print(results)

    draw_styled_landmarks(image,results)

    #Prediction logic code
    keypoints = extract_keypoints(results)
    sequence.insert(keypoints)
    sequence = sequence[:-30]

    if len(sequence) ==30:
      res = model.predict(np.expand_dims(sequence,axis=0))[0]
      print(actions[np.argmax(res)])


      #Visulaisation prediction logic
    if res[np.argmax(res)] > threshold:
      if len(sentence) > 0:
        if actions[np.argmax(res)] != sentence[-1]:
          sentence.append(actions[np.argmax(res)])
      else:
          sentence.append(actions[np.argmax(res)])
    if len(sentence) > 5:
      sentence = sentence[-5:]

    #Visualise Probability
    image = prob_viz(res,actions,image,actions)
    cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
    cv2.putText(image, ''.join(sentence),(3,30),
                  cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('OpenCV Feed',image)

    if cv2.waitKey(10) & 0xFF ==ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>


TypeError: insert expected 2 arguments, got 1

In [76]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti